In [1]:
from typing import Callable, Union

import torch
import torch.nn as nn
from torch.nn.modules.module import _grad_t
from torch.utils.hooks import RemovableHandle
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision
import numpy as np
import pandas as pd 

In [2]:
train_dataset = MNIST( root='./data', train = True , download = True, transform = transforms.ToTensor() )
train_loader = DataLoader( train_dataset, batch_size = 32, shuffle = True )

test_dataset = MNIST( root='./data', train = False, download = False, transform = transforms.ToTensor() )
test_loader = DataLoader( test_dataset, batch_size = 32, shuffle = False )

RNN的输入: seq_len, batch_size, input_size( feature_len ) -> batch_size -> dim = 1, 用batch_first = True把batch_size调到最前面

In [ ]:
class LSTM( nn.Module ):
    def __init__( self, input_dim, hidden_dim, layer_dim, output_dim ):
        def __init__( self, input_dim, hidden_dim, output_dim ):
            super( LSTM, self ).__init__()
            self.hidden_dim = hidden_dim
            self.layer_dim = layer_dim
            self.rnn = nn.RNN( input_dim, hidden_dim, layer_dim, batch_first = True, nonlinearity = 'relu' )
            self.fc = nn.Linear( hidden_dim, output_dim )
            
            
        def forward( self, x ):
            # 初始化隐藏层状态 -> layer_dim, batch_size, hidden_dim     x.size( 0 ) -> batch_size
            h0 = torch.zeros( self.layer_dim, x.size( 0 ), self.hidden_dim ).requires_grad_()
            out, hn = self.rnn( x, h0.detach() )                # 分离隐藏状态，避免梯度爆炸
            
            out = self.fc( out[ :, -1, : ] )
            return out

In [ ]:
input_dim = 28          # 输入的图片都是28×28的
hidden_dim = 100        # 隐藏层的维度
layer_dim = 2           # 2层RNN
output_dim = 10         # 输出维度 -> 一共有十种数字

lstm = LSTM( input_dim, hidden_dim, layer_dim, output_dim )
loss_f = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD( lstm.parameters(), lr = 0.1 )